<a href="https://colab.research.google.com/github/anaguilarar/gee_satellite_data/blob/master/Download%20Satellite%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall pyproj
!pip install pyproj
!pip install wget
!pip install geehydro
!pip install geopandas

In [ ]:
!git clone https://github.com/anaguilarar/gee_satellite_data.git

In [ ]:
import os
os.chdir("gee_satellite_data")

In [1]:
import ee
#ee.Authenticate()

In [2]:

import pandas as pd
from scripts import gee_satellite_data
from scripts import gee_functions
from scripts import gis_functions
from scripts import general_functions

## Download Sentinel 2A: TOA level

Before downloading data, the code internally removes hose areas covered by shadows and clouds. 

In [3]:
se2_toa = gee_satellite_data.get_gee_data("2016-01-01",
                                            "2016-09-30", 
                                            "data/col_t3.shp", 
                                            "sentinel2_toa",
                                         cloud_percentage= 70)

As a result, we get a list which coantains the images dates available for quering period. Besides a cover percentage feature is calculted, it reffers to ratio of image clean area respect to our area of interest.

In [4]:
se2_toa.summary

,dates,cover_percentage
0,2016-01-10 00:00:00,63.481518
1,2016-01-30 00:00:00,85.445388
2,2016-02-09 00:00:00,75.885103
3,2016-02-19 00:00:00,96.685545
4,2016-03-10 00:00:00,49.261255
5,2016-04-19 00:00:00,94.757381
6,2016-04-29 00:00:00,96.297396
7,2016-05-19 10:31:18,60.853345
8,2016-06-08 00:00:00,0.419447
9,2016-06-18 00:00:00,97.977475


In [5]:
truecolorParams = {'gamma': 1.3, 
                   'min': 0,
                   'max': 0.3,
                   'bands': ['B4','B3','B2']
                   }

imagetoplot = se2_toa.image_collection.toList(se2_toa.image_collection.size()).get(12)

gee_satellite_data.plot_eeimage(imagetoplot, truecolorParams, se2_toa.geometry, zoom = 11)

In [6]:
se2_toa.add_vi_layer("ndvi")

## Superpixel: Image Segmentation
In the next section we use the segmetation Simple Non-Iterative Clustering (SNIC) module from GEE (https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic) to create image segments. 
Each of these segments will be treated as a superpixel. 

We will use each of the superpixels as the input features for the radar images smoothing. we are going trough following steps, to get a radar images smoothing using superpixels from optical images:
 * Apply image segmention to optical images
 * Get polygons from superpixels
 * reduce each radar image using the oplygons


In [14]:
cover_percentage = 94
## filter collection by cover percentage
imgcollection = filtering_bycoverpercentage(se2_toa, cover_percentage)
## get image segmentation using SNIC method
snic = gee_snic(imgcollection,se2_toa._ee_sp )
### from image segments to polygons
polygons_fromseg = raster_to_polygons(snic, imgcollection.select(["B4"]
                                        ).median().clip(se2_toa._ee_sp), se2_toa._ee_sp, scale = 10)



total images: 16 
total images after cover filter: 8


### Sentinel 1 
The instrument onboard the Sentinel-1 satellites transmits in one polarisation (V) and measures in two (V and H), giving the VV and VH bands. The first band in each data file is the signal measured in the same polarisation as the transmission (VV) and the second is measured in the orthogonal polarisation (VH).


In [21]:
s1_imagery = gee_satellite_data.get_gee_data("2016-01-01",
                                            "2016-09-30", 
                                            "data/col_t3.shp", 
                                            "sentinel1", bands = ['VV'])
s1_imagery.summary

Due to Sentinel 1 mission has two satellites (a and b), it can orbit the earth in a different orientation (Ascending and Descending), the  6 days data can be summarised averaging over a time window. 

In [24]:
s1_imagery.reduce_collection_by_days(10)
s1_imagery._dates_reduced

0    2016-01-21 05:42:59
1    2016-02-07 18:13:28
2    2016-02-14 05:42:59
3    2016-03-02 18:13:28
4    2016-03-09 05:42:58
5    2016-03-29 23:58:12
6    2016-04-19 18:13:29
7    2016-04-26 05:43:00
8    2016-05-16 23:58:16
9    2016-06-06 18:13:32
10   2016-06-13 05:43:03
11   2016-06-30 18:13:33
12   2016-07-07 05:43:05
13   2016-07-27 23:58:20
14   2016-08-17 18:13:36
15   2016-08-24 05:43:07
dtype: datetime64[ns]

In [25]:
s1datasuperpixel= s1_imagery.image_collection.map(lambda image:
                                gee_functions.reduce_image_to_superpixel(image, polygons_fromseg))

Once the superpixels has been obtained, we are going to summary our s1 collection in percentiles

In [27]:
abundance_perc = s1datasuperpixel.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95]))
abundance_perc_orig = s1_imagery.image_collection.reduce(ee.Reducer.percentile([5,10, 25, 50, 75 , 90, 95]))


Finally we compare both results

In [ ]:
import folium 
display = ee.Image(0).updateMask(0).paint(polygons_fromseg, '000000', 3);

centergeometry = gis_functions.geometry_center(s1_imagery.geometry)

Map = folium.Map(location=[centergeometry[1],
                               centergeometry[0]], zoom_start=12)

Map.addLayer(ee.Image(abundance_perc).select(['first_p10', 'first_p50', 'first_p90']), {'gamma': 1.3, 'min':-25, 'max':0}, 'superpixel');
Map.addLayer(display, {}, 'vectors');
Map.addLayer(ee.Image(abundance_perc_orig).select(['VV_p10', 'VV_p50', 'VV_p90']), {'gamma': 1.3, 'min':-25, 'max':0}, 'orig');

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

Map